<a href="https://colab.research.google.com/github/samuramirez/cellmigration/blob/master/VAMPIRE_on_colab_ByRegion_3Regions_AllOneTreatment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import auth
#This allows SDK to see and edit Google Drive files
#SDK is required to interact with GCP
auth.authenticate_user()

In [3]:
#Get from Drive the folder libraries
!cp -r /content/drive/MyDrive/SourceCode /content

from SourceCode.mainbody import mainbody
from SourceCode.getboundary import getboundary

In [4]:
import csv
import ntpath
import re
import os
import numpy as np

# Copy segmented, labeled images from GCP to local directory

In [5]:
treatment_path = 'cellmbucket/analysis/controls_paths_UPDATE.txt' #@param {type:"string"}

#read treatment locations file
!gsutil cp gs://{treatment_path} ./
treatment_file = ntpath.basename(treatment_path)
treatment_obj = open(treatment_file, "r")
treatment_locs = treatment_obj.readlines()

Copying gs://cellmbucket/analysis/controls_paths_UPDATE.txt...
/ [1 files][  1.2 KiB/  1.2 KiB]                                                
Operation completed over 1 objects/1.2 KiB.                                      


In [6]:
treatment_textfile_path = 'cellmbucket/analysis/controls_textfiles_paths.txt' #@param {type:"string"}

#read treatment text locations file
!gsutil cp gs://{treatment_textfile_path} ./
treatment_text_file = ntpath.basename(treatment_textfile_path)
treatment_text_obj = open(treatment_text_file, "r")
treatment_text_locs = treatment_text_obj.readlines()

Copying gs://cellmbucket/analysis/controls_textfiles_paths.txt...
/ [1 files][  1.0 KiB/  1.0 KiB]                                                
Operation completed over 1 objects/1.0 KiB.                                      


In [7]:
treatment_locs

['controls\n',
 'cellmbucket/analysis_042221/06032020-JR20-control\n',
 'cellmbucket/analysis_pri_050521/03122021-JR20P14-control-sample5\n',
 'cellmbucket/analysis_pri_050521/03122021-JR20P14-control\n',
 'cellmbucket/analysis_sam_042821/031521_control_sample3\n',
 'cellmbucket/analysis_sam_042821/031521_control_sample4\n',
 'cellmbucket/analysis_sam_042821/031521_control_sample5\n',
 '\n',
 'cellmbucket/analysis_eliz_05042021/trial1/05042021_ARPC2_KO_sample6\n',
 'cellmbucket/analysis_eliz_05042021/trial3/05112021_ARPC2_KO_sample3\n',
 'cellmbucket/analysis_eliz_05042021/trial4/05202021_ARPC2_KO_sample4\n',
 'cellmbucket/analysis_eliz_05042021/trial5/060622021_ARPC2_KO_sample3\n',
 'cellmbucket/analysis_eliz_05042021/trial5/060622021_ARPC2_KO_sample4\n',
 'cellmbucket/analysis_eliz_05262021/trial1/052622021_Fascin_KO_sample3\n',
 'cellmbucket/analysis_eliz_05262021/trial2/052822021_Fascin_KO_sample3\n',
 'cellmbucket/analysis_eliz_05262021/trial3/060222021_Fascin_KO_sample3\n',
 '\n'

In [8]:
treatment_text_locs

['controls-text_file_locations\n',
 'cellmbucket/06032020-JR20-control-masks/sample1_info.txt\n',
 'cellmbucket/03122021-JR20P14-control-masks/sample5_info.txt\n',
 'cellmbucket/03122021-JR20P14-control-masks/sample6_info.txt\n',
 'cellmbucket/03152021-JR20P6-control-masks/sample3_info.txt\n',
 'cellmbucket/03152021-JR20P6-control-masks/sample4_info.txt\n',
 'cellmbucket/03152021-JR20P6-control-masks/sample5_info.txt\n',
 '\n',
 'cellmbucket/05042021_ARPC2_KO/trial1/sample6_info.txt\n',
 'cellmbucket/05042021_ARPC2_KO/trial3/sample3_info.txt\n',
 'cellmbucket/05042021_ARPC2_KO/trial4/sample4_info.txt\n',
 'cellmbucket/05042021_ARPC2_KO/trial5/sample3_info.txt\n',
 'cellmbucket/05042021_ARPC2_KO/trial5/sample4_info.txt\n',
 'cellmbucket/05262021_Fascin_KO/trial1/sample3_info.txt\n',
 'cellmbucket/05262021_Fascin_KO/trial2/sample3_info.txt\n',
 'cellmbucket/05262021_Fascin_KO/trial3/sample3_info.txt\n',
 '\n',
 'cellmbucket/09162021/09162021_info.txt\n',
 '\n',
 'cellmbucket/controls_201

In [9]:
#get all files full paths. Some data has the same name but is stored in different folder
file_paths =[]
for i in range(1,len(treatment_locs)):
  if len(treatment_locs[i].strip("\n")) > 0: #if line contains data
    file_paths.append(treatment_locs[i].strip("\n")+'_trackedmasks') 

In [10]:
#get all files full paths. Some data has the same name but is stored in different folder
textfile_paths =[]
for i in range(1,len(treatment_text_locs)):
  if len(treatment_text_locs[i].strip("\n")) > 0: #if line contains data
    textfile_paths.append(treatment_text_locs[i].strip("\n")) 

In [11]:
file_paths

['cellmbucket/analysis_042221/06032020-JR20-control_trackedmasks',
 'cellmbucket/analysis_pri_050521/03122021-JR20P14-control-sample5_trackedmasks',
 'cellmbucket/analysis_pri_050521/03122021-JR20P14-control_trackedmasks',
 'cellmbucket/analysis_sam_042821/031521_control_sample3_trackedmasks',
 'cellmbucket/analysis_sam_042821/031521_control_sample4_trackedmasks',
 'cellmbucket/analysis_sam_042821/031521_control_sample5_trackedmasks',
 'cellmbucket/analysis_eliz_05042021/trial1/05042021_ARPC2_KO_sample6_trackedmasks',
 'cellmbucket/analysis_eliz_05042021/trial3/05112021_ARPC2_KO_sample3_trackedmasks',
 'cellmbucket/analysis_eliz_05042021/trial4/05202021_ARPC2_KO_sample4_trackedmasks',
 'cellmbucket/analysis_eliz_05042021/trial5/060622021_ARPC2_KO_sample3_trackedmasks',
 'cellmbucket/analysis_eliz_05042021/trial5/060622021_ARPC2_KO_sample4_trackedmasks',
 'cellmbucket/analysis_eliz_05262021/trial1/052622021_Fascin_KO_sample3_trackedmasks',
 'cellmbucket/analysis_eliz_05262021/trial2/052

In [12]:
textfile_paths

['cellmbucket/06032020-JR20-control-masks/sample1_info.txt',
 'cellmbucket/03122021-JR20P14-control-masks/sample5_info.txt',
 'cellmbucket/03122021-JR20P14-control-masks/sample6_info.txt',
 'cellmbucket/03152021-JR20P6-control-masks/sample3_info.txt',
 'cellmbucket/03152021-JR20P6-control-masks/sample4_info.txt',
 'cellmbucket/03152021-JR20P6-control-masks/sample5_info.txt',
 'cellmbucket/05042021_ARPC2_KO/trial1/sample6_info.txt',
 'cellmbucket/05042021_ARPC2_KO/trial3/sample3_info.txt',
 'cellmbucket/05042021_ARPC2_KO/trial4/sample4_info.txt',
 'cellmbucket/05042021_ARPC2_KO/trial5/sample3_info.txt',
 'cellmbucket/05042021_ARPC2_KO/trial5/sample4_info.txt',
 'cellmbucket/05262021_Fascin_KO/trial1/sample3_info.txt',
 'cellmbucket/05262021_Fascin_KO/trial2/sample3_info.txt',
 'cellmbucket/05262021_Fascin_KO/trial3/sample3_info.txt',
 'cellmbucket/09162021/09162021_info.txt',
 'cellmbucket/controls_2019/062719/sample1_info.txt',
 'cellmbucket/controls_2019/062719/sample2_info.txt',
 'ce

In [13]:
for count,maskspath in enumerate(file_paths):
  !gsutil cp gs://{maskspath}.zip /content
  !unzip {ntpath.basename(maskspath)}.zip

  text_file = textfile_paths[count]
  !gsutil cp gs://{text_file} ./

  strings=os.listdir(ntpath.basename(maskspath))
  regions = ['stiff', 'steep1', 'steep2', 'soft']
  regions_obj = open(ntpath.basename(text_file), "r")
  lines_to_list = regions_obj.readlines()
  for word in regions:
    for line in lines_to_list:
      if word in line:
        split = line.split('-')
        start = int(split[0].lstrip(word))
        end = int(split[1].lstrip('/n'))
        if os.path.exists('/content/' + word) == False:
          !mkdir {word}
        substring=list(range(start, end+1))
        substring=['_s'+str(i)+'_' for i in substring]
        strings_with_substring = [string for string in strings if any(xs in string for xs in substring)]
        for name in strings_with_substring:
          !cp /content/{ntpath.basename(maskspath)}/{name} /content/{word}

!mkdir steep
!cp -a /content/steep1/. /content/steep
!cp -a /content/steep2/. /content/steep 
!rm -r steep1 
!rm -r steep2

Streaming output truncated to the last 5000 lines.
  inflating: controls_2019_062719_sample2_trackedmasks/062719_Sample2_w1DIC_s11_t44.TIF  
  inflating: controls_2019_062719_sample2_trackedmasks/062719_Sample2_w1DIC_s18_t87.TIF  
  inflating: controls_2019_062719_sample2_trackedmasks/062719_Sample2_w1DIC_s6_t120.TIF  
  inflating: controls_2019_062719_sample2_trackedmasks/062719_Sample2_w1DIC_s3_t73.TIF  
  inflating: controls_2019_062719_sample2_trackedmasks/062719_Sample2_w1DIC_s17_t84.TIF  
  inflating: controls_2019_062719_sample2_trackedmasks/062719_Sample2_w1DIC_s13_t71.TIF  
  inflating: controls_2019_062719_sample2_trackedmasks/062719_Sample2_w1DIC_s16_t117.TIF  
  inflating: controls_2019_062719_sample2_trackedmasks/062719_Sample2_w1DIC_s1_t105.TIF  
  inflating: controls_2019_062719_sample2_trackedmasks/062719_Sample2_w1DIC_s26_t13.TIF  
  inflating: controls_2019_062719_sample2_trackedmasks/062719_Sample2_w1DIC_s14_t20.TIF  
  inflating: controls_2019_062719_sample2_tracked

##Save images separated by region on GCS

In [14]:
!zip -r controls.zip soft steep stiff 

Streaming output truncated to the last 5000 lines.
  adding: stiff/060222021_Sample3_w1DIC_s4_t59.TIF (deflated 100%)
  adding: stiff/052622021_Sample3_w1DIC_s4_t156.TIF (deflated 100%)
  adding: stiff/062719_Sample1_w1DIC_s6_t22.TIF (deflated 100%)
  adding: stiff/060622021_Sample3_w1DIC_s5_t75.TIF (deflated 100%)
  adding: stiff/sample6_s3_t50.TIF (deflated 100%)
  adding: stiff/062719_Sample1_w1DIC_s3_t43.TIF (deflated 100%)
  adding: stiff/052822021_Sample3_w1DIC_s8_t2.TIF (deflated 100%)
  adding: stiff/060622021_Sample3_w1DIC_s8_t58.TIF (deflated 100%)
  adding: stiff/062719_Sample1_w1DIC_s6_t68.TIF (deflated 100%)
  adding: stiff/062719_Sample2_w1DIC_s1_t99.TIF (deflated 100%)
  adding: stiff/060622021_Sample4_w1DIC_s7_t124.TIF (deflated 100%)
  adding: stiff/052622021_Sample3_w1DIC_s3_t153.TIF (deflated 100%)
  adding: stiff/060622021_Sample3_w1DIC_s3_t33.TIF (deflated 100%)
  adding: stiff/052822021_Sample3_w1DIC_s2_t55.TIF (deflated 100%)
  adding: stiff/sample4_s4_t3.TIF (de

In [15]:
!gsutil cp /content/controls.zip gs://cellmbucket/analysis

Copying file:///content/controls.zip [Content-Type=application/zip]...
-
Operation completed over 1 objects/35.8 MiB.                                     


# Make csv file that contains paths to segmented, labeled images in local directory with descriptions for each set

In [16]:
csv_fields=['set ID', 'condition', 'set location', 'tag', 'note']

In [17]:
#@markdown Parameters for the csv that indicate where the masks are located

#@markdown Row index number
setID1 = '1' #@param {type:"string"}
setID2 = '2' #@param {type:"string"}
setID3 = '3' #@param {type:"string"}
#@markdown Description of an image set
condition1 = 'stiff' #@param {type:"string"}
condition2 = 'steep' #@param {type:"string"}
condition3 = 'soft' #@param {type:"string"}
#@markdown The path of the folder containing the segmented images
set_location1 = '/content/stiff' #@param {type:"string"}
set_location2 = '/content/steep' #@param {type:"string"}
set_location3 = '/content/soft' #@param {type:"string"}
#@markdown String of text where only segmented images in the set location with filenames containing the tag will be analyzed
tag1 = 'DIC' #@param {type:"string"}
tag2 = 'DIC' #@param {type:"string"}
tag3 = 'DIC' #@param {type:"string"}
#@markdown Any information about the image sets used only for the user's record. This is not used in the analysis
note1 = 'all_controls' #@param {type:"string"}
note2 = 'all_controls' #@param {type:"string"}
note3 = 'all_controls' #@param {type:"string"}
#@markdown Name of the csv file that is created
csv_name = 'segmented_image_sets_to_build_model_colab' #@param {type:"string"}

In [18]:
rows=[[setID1,
  condition1,
  set_location1,
  tag1,
  note1],
 [setID2,
  condition2,
  set_location2,
  tag2,
  note2],
  [setID3,
  condition3,
  set_location3,
  tag3,
  note3]]

In [19]:
csv_filename = '/content/' + csv_name + '.csv'

In [20]:
with open(csv_filename, 'w') as csvfile:
    # creating a csv writer object
    csvwriter = csv.writer(csvfile)
      
    # writing the fields
    csvwriter.writerow(csv_fields)
      
    # writing the data rows
    csvwriter.writerows(rows)

# Specify VAMPIRE model parameters

In [21]:
entries={}

In [22]:
fields = (
        'Build Model', 'Image sets to build', 'Number of coordinates', 'Number of principal components', 'Number of shape modes', 'Model output folder',
        'Model name', 'Image sets to apply', 'Model to apply', 'Result output folder')

In [31]:
#@markdown Parameters for VAMPIRE model

#@markdown Build model (True or False)
Build_Model = False #@param {type:"boolean"}
#@markdown Path to csv file to load that contains path to masks
Image_sets_to_build = '/content/segmented_image_sets_to_build_model_colab.csv' #@param {type:"string"}
#@markdown Number of coordinates used to describe cell shape
Number_of_coordinates = '50' #@param {type:"string"}
#@markdown Number of principal components to use for PCA
pcnum = '10' #@param {type:"string"}
#@markdown Number of clusters to form to describe cell shapes
Number_of_shape_modes = '10' #@param {type:"string"}
#@markdown Path to location where output data is saved
Model_output_folder = '/content' #@param {type:"string"}
#@markdown Name of folder where output will be saved (and name of the model pickle file)
Model_name = '50Coords10PC10ShapeModes_3Regions_allcontrols' #@param {type:"string"}
#@markdown Apply model

#@markdown Only complete the below entries if you are applying the model

#@markdown Path to csv file to load that contains path to masks
Image_sets_to_apply = '/content/segmented_image_sets_to_build_model_colab.csv' #@param {type:"string"}
#@markdown Path to pickle model file 
Model_to_apply = '/content/50Coords10PC10ShapeModes_3Regions_allcontrols/50Coords10PC10ShapeModes_3Regions_allcontrols.pickle' #@param {type:"string"}
#@markdown Path to location where outpur data is saved (usually the folder that was created with Build Model)
Result_output_folder = '/content/50Coords10PC10ShapeModes_3Regions_allcontrols' #@param {type:"string"}


In [32]:
ents = [Build_Model, Image_sets_to_build, Number_of_coordinates, pcnum, Number_of_shape_modes, Model_output_folder, Model_name, Image_sets_to_apply, Model_to_apply, Result_output_folder]

for count,field in enumerate(fields):
  entries[field] = ents[count]

In [33]:
# Make folders in local directory where figures and model pickle file will be saved 

if Build_Model:
  foldername = Model_output_folder + '/' + Model_name
  !mkdir {foldername}
  newfolder = foldername + '/' + 'Example model figures'
  !mkdir '{newfolder}'
else:
  foldername = Result_output_folder + '/' + 'Result based on ' + ntpath.basename(Result_output_folder)
  !mkdir '{foldername}'

In [34]:
entries

{'Build Model': False,
 'Image sets to apply': '/content/segmented_image_sets_to_build_model_colab.csv',
 'Image sets to build': '/content/segmented_image_sets_to_build_model_colab.csv',
 'Model name': '50Coords10PC10ShapeModes_3Regions_allcontrols',
 'Model output folder': '/content',
 'Model to apply': '/content/50Coords10PC10ShapeModes_3Regions_allcontrols/50Coords10PC10ShapeModes_3Regions_allcontrols.pickle',
 'Number of coordinates': '50',
 'Number of principal components': '10',
 'Number of shape modes': '10',
 'Result output folder': '/content/50Coords10PC10ShapeModes_3Regions_allcontrols'}

In [35]:
def Model(entries, buildModel):
    if buildModel:
        csv = entries['Image sets to build']
        clnum = entries['Number of shape modes']
        outpth = entries['Model output folder']  # name
        getboundary(csv, outpth, entries, buildModel)  # create registry csv and boundary stack
        mainbody(buildModel, csv, entries, outpth, clnum)
    else:
        csv = entries['Image sets to apply']
        outpth = entries['Result output folder']
        clnum = None
        getboundary(csv, outpth, entries, buildModel)  # create registry csv and boundary stack
        mainbody(buildModel, csv, entries, outpth, clnum)


def vampire():
    Model(entries, Build_Model)

# Run the VAMPIRE analysis

In [36]:
vampire()

## getboundary.py
registry or boundary already exist
registry or boundary already exist
registry or boundary already exist
## main.py
## bdreg.py
applying model
available cpu cores :  2
For parallel of bdreg, elapsed time is 67.8967490196228seconds...
## pca_bdreg.py
For PCA bdreg, elapsed time is 0.01994609832763672seconds...
# clusterSM
For cluster, elapsed time is 1.042647361755371seconds...
## update_csv.py
## bdreg.py
applying model
available cpu cores :  2
For parallel of bdreg, elapsed time is 130.99122262001038seconds...
## pca_bdreg.py
For PCA bdreg, elapsed time is 0.036180973052978516seconds...
# clusterSM
For cluster, elapsed time is 1.1896216869354248seconds...
## update_csv.py
## bdreg.py
applying model
available cpu cores :  2
For parallel of bdreg, elapsed time is 37.35017538070679seconds...
## pca_bdreg.py
For PCA bdreg, elapsed time is 0.008047819137573242seconds...
# clusterSM
For cluster, elapsed time is 0.9618759155273438seconds...
## update_csv.py


# Copy files generated from VAMPIRE analysis (csv files of shape features, model pickle file, pickle files of boundary coordinates, and figures) to GCP bucket

In [37]:
#@markdown Where to save VAMPIRE files in GCP bucket
GCP_path = 'cellmbucket/VAMPIRE_analysis/all_controls' #@param {type:"string"}
#@markdown Name of folder that will contain all generated files from the VAMPIRE analysis in GCP bucket
output_folder = '11152021' #@param {type:"string"}

In [38]:
output_data = Model_output_folder + '/' + Model_name

output_path = GCP_path + '/' + output_folder + '/'

!gsutil cp -r '{output_data}'  gs://{output_path}

Copying file:///content/50Coords10PC10ShapeModes_3Regions_allcontrols/50Coords10PC10ShapeModes_3Regions_allcontrols.pickle [Content-Type=application/octet-stream]...
Copying file:///content/50Coords10PC10ShapeModes_3Regions_allcontrols/VAMPIRE datasheet steep.csv [Content-Type=text/csv]...
Copying file:///content/50Coords10PC10ShapeModes_3Regions_allcontrols/steep_boundary_coordinate_stack.pickle [Content-Type=application/octet-stream]...
Copying file:///content/50Coords10PC10ShapeModes_3Regions_allcontrols/stiff_boundary_coordinate_stack.pickle [Content-Type=application/octet-stream]...
| [4 files][ 61.1 MiB/ 61.1 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file:///content/50Coords10PC10ShapeModes_3Regions_allcontrols/so

## This is just code to copy things from google cloud bucket to here to zip and download locally

In [ ]:
!gsutil cp -r gs://cellmbucket/analysis_eliz_05262021/trial1/052622021_Fascin_KO_VAMPIRE_analysis/50Coords10ShapeModes_AllRegionsSample4 ./

Copying gs://cellmbucket/analysis_eliz_05262021/trial1/052622021_Fascin_KO_VAMPIRE_analysis/50Coords10ShapeModes_AllRegionsSample4/50Coords10ShapeModes_AllRegionsSample4.pickle...
Copying gs://cellmbucket/analysis_eliz_05262021/trial1/052622021_Fascin_KO_VAMPIRE_analysis/50Coords10ShapeModes_AllRegionsSample4/Example model figures/Registered objects.png...
Copying gs://cellmbucket/analysis_eliz_05262021/trial1/052622021_Fascin_KO_VAMPIRE_analysis/50Coords10ShapeModes_AllRegionsSample4/Example model figures/Shape mode dendrogram.png...
Copying gs://cellmbucket/analysis_eliz_05262021/trial1/052622021_Fascin_KO_VAMPIRE_analysis/50Coords10ShapeModes_AllRegionsSample4/Result based on 50Coords10ShapeModes_AllRegionsSample4/Shape mode distribution_1_stiff.png...
- [4 files][  1.4 MiB/  1.4 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m sect

In [ ]:
!zip -r "/content/50Coords10ShapeModes_AllRegionsSample4.zip" "/content/50Coords10ShapeModes_AllRegionsSample4"

  adding: content/50Coords10ShapeModes_AllRegionsSample4/ (stored 0%)
  adding: content/50Coords10ShapeModes_AllRegionsSample4/stiff_boundary_coordinate_stack.pickle (deflated 90%)
  adding: content/50Coords10ShapeModes_AllRegionsSample4/Result based on 50Coords10ShapeModes_AllRegionsSample4/ (stored 0%)
  adding: content/50Coords10ShapeModes_AllRegionsSample4/Result based on 50Coords10ShapeModes_AllRegionsSample4/Shape mode distribution_3_steep2.png (deflated 9%)
  adding: content/50Coords10ShapeModes_AllRegionsSample4/Result based on 50Coords10ShapeModes_AllRegionsSample4/Shape mode distribution_2_steep1.png (deflated 8%)
  adding: content/50Coords10ShapeModes_AllRegionsSample4/Result based on 50Coords10ShapeModes_AllRegionsSample4/Shape mode distribution_4_soft.png (deflated 9%)
  adding: content/50Coords10ShapeModes_AllRegionsSample4/Result based on 50Coords10ShapeModes_AllRegionsSample4/Shape mode distribution_1_stiff.png (deflated 9%)
  adding: content/50Coords10ShapeModes_AllReg